# Relatório Técnico TISS - Análise Descritiva

## Espírito Santo - 1º Trimestre de 2023

Este relatório apresenta uma análise descritiva dos dados da [translate:Troca de Informação de Saúde Suplementar (TISS)], com recorte para o estado do Espírito Santo no primeiro trimestre de 2023.

O objetivo principal é entender a estrutura principal do banco de dados, explorar as variáveis e demonstrar algumas análises descritivas por meio de tabelas e gráficos.

---

# Configuração do Ambiente

In [ ]:
# Instalar pacotes se necessário
# install.packages(c('tidyverse', 'gt', 'ggplot2', 'kableExtra', 'viridis', 'scales'))

# Carregar bibliotecas necessárias
library(tidyverse)    # Manipulação de dados
library(gt)           # Tabelas elegantes
library(ggplot2)      # Visualizações
library(kableExtra)   # Tabelas HTML
library(viridis)      # Paletas de cores
library(scales)       # Formatação de escalas
library(stringr)      # Manipulação de strings

In [ ]:
# Carregar os dados
# Nota: substitua o caminho conforme necessário
# dados_1T_2023 <- read_csv('caminho/para/dados_1T_2023.csv')

# Para este exemplo, estamos assumindo que os dados já estão carregados
cat("Dados carregados com sucesso!\\n")

# 1. Análise da Demanda Hospitalar

## Internações por Tipo e Caráter

A compreensão da demanda hospitalar inicia-se pela distinção entre os atendimentos programados (eletivos) e os de caráter imediato (urgência/emergência). A tabela a seguir cruza essa informação com o tipo de internação.

In [ ]:
######################################
# Tabela de Internações por Tipo e Caráter
######################################

agrupador <- dados_1T_2023 %>%
  group_by(ID_EVENTO_ATENCAO_SAUDE) %>%
  summarise(
    CD_CARATER_ATENDIMENTO = max(CD_CARATER_ATENDIMENTO),
    CD_TIPO_INTERNACAO = max(CD_TIPO_INTERNACAO)
  )

tabela_final <- agrupador %>%
  filter(CD_CARATER_ATENDIMENTO == 1 | CD_CARATER_ATENDIMENTO == 2) %>%
  mutate(
    Carater = case_when(
      CD_CARATER_ATENDIMENTO == 1 ~ "Eletiva",
      TRUE ~ "Urgência/Emergência"
    ),
    Tipo = case_when(
      CD_TIPO_INTERNACAO == 1 ~ "Clínica",
      CD_TIPO_INTERNACAO == 2 ~ "Cirúrgica",
      CD_TIPO_INTERNACAO == 3 ~ "Obstétrica",
      CD_TIPO_INTERNACAO == 4 ~ "Pediátrica",
      TRUE ~ "Psiquiátrica"
    )
  ) %>%
  count(Carater, Tipo, name = "Internações") %>%
  pivot_wider(
    names_from = Tipo,
    values_from = Internações,
    values_fill = 0
  ) %>%
  rename(`Caráter de Atendimento` = Carater)

# Exibição com gt
tabela_final %>%
  gt() %>%
  tab_header(
    title = "Matriz de Contagem de Internações",
    subtitle = "Análise cruzada entre Caráter e Tipo de Internação."
  ) %>%
  tab_style(
    style = cell_text(weight = "bold"),
    locations = cells_title()
  ) %>%
  tab_style(
    style = cell_text(size = px(13.5)),
    locations = cells_body()
  ) %>%
  cols_width(everything() ~ px(132)) %>%
  fmt_integer(locale = "pt_BR") %>%
  tab_source_note(
    source_note = "Fonte: Dados da TISS (Amostra do Espírito Santo, 1º Trimestre de 2023)"
  ) %>%
  tab_options(source_notes.font.size = px(10))

In [ ]:
######################################
# Tabela de Proporções
######################################

total_base <- nrow(agrupador)

tabela_proporcao <- tabela_final %>%
  mutate(across(
    where(is.numeric),
    ~ paste0(
      format(
        round((. / total_base) * 100, 1),
        decimal.mark = ",",
        trim = TRUE
      ),
      "%"
    )
  ))

# Exibição
tabela_proporcao %>%
  gt() %>%
  tab_header(
    title = "Matriz de Proporção de Internações",
    subtitle = "Análise cruzada entre Caráter e Tipo de Internação"
  ) %>%
  tab_style(
    style = cell_text(weight = "bold"),
    locations = cells_title()
  ) %>%
  tab_style(
    style = cell_text(size = px(13.5)),
    locations = cells_body()
  ) %>%
  cols_width(everything() ~ px(132)) %>%
  tab_source_note(
    source_note = "Fonte: Dados da TISS (Amostra do Espírito Santo, 1º Trimestre de 2023)"
  ) %>%
  tab_options(source_notes.font.size = px(10))

## Fluxo de Pacientes e Diagnósticos Principais

Dois indicadores críticos para a regulação do setor são o fluxo de pacientes entre municípios e as principais causas de internação (CID-10).

In [ ]:
######################################
# Cálculo de Migração
######################################

agrupador_geo <- dados_1T_2023 %>%
  group_by(ID_EVENTO_ATENCAO_SAUDE) %>%
  summarise(
    CD_MUNICIPIO_BENEFICIARIO = max(CD_MUNICIPIO_BENEFICIARIO),
    CD_MUNICIPIO_PRESTADOR = max(CD_MUNICIPIO_PRESTADOR)
  ) %>%
  mutate(
    fluxo_interno = if_else(
      CD_MUNICIPIO_BENEFICIARIO == CD_MUNICIPIO_PRESTADOR, 1, 0
    )
  )

taxa_migracao <- (1 - mean(agrupador_geo$fluxo_interno, na.rm = TRUE)) * 100

cat(sprintf("Taxa de Migração (fluxo entre municípios): %.2f%%\\n", taxa_migracao))

In [ ]:
######################################
# Tabela de Internações por Diagnóstico (CID-10)
######################################

rm(agrupador)

agrupador <- dados_1T_2023 %>%
  select(ID_EVENTO_ATENCAO_SAUDE, CD_CARATER_ATENDIMENTO, ID_CID10) %>%
  group_by(ID_EVENTO_ATENCAO_SAUDE) %>%
  summarise(
    ID_CID10 = max(ID_CID10),
    CD_CARATER_ATENDIMENTO = max(CD_CARATER_ATENDIMENTO)
  ) %>%
  filter(ID_CID10 %in% c("R", "I", "J", "N", "O")) %>%
  filter(CD_CARATER_ATENDIMENTO %in% c(1, 2)) %>%
  mutate(
    Carater = case_when(
      CD_CARATER_ATENDIMENTO == 1 ~ "Eletiva",
      TRUE ~ "Urgência/Emergência"
    ),
    Diagnostico = case_when(
      ID_CID10 == 'R' ~ "Achados anormais em exames",
      ID_CID10 == 'I' ~ "Aparelho Circulatório",
      ID_CID10 == 'J' ~ "Aparelho Respiratório",
      ID_CID10 == 'N' ~ "Aparelho Geniturinário",
      TRUE ~ "Relacionados à gravidez"
    )
  ) %>%
  count(Carater, Diagnostico, name = "Internações") %>%
  pivot_wider(
    names_from = Diagnostico,
    values_from = Internações,
    values_fill = 0
  ) %>%
  rename(`Caráter de Atendimento` = Carater)

tabela_final <- agrupador

# Exibição
tabela_final %>%
  gt() %>%
  tab_header(
    title = "Matriz de Contagem de Internações",
    subtitle = "Análise cruzada entre Caráter e Tipos de Diagnóstico"
  ) %>%
  tab_style(
    style = cell_text(weight = "bold"),
    locations = cells_title()
  ) %>%
  tab_style(
    style = cell_text(size = px(13.5)),
    locations = cells_body()
  ) %>%
  cols_width(everything() ~ px(132)) %>%
  fmt_integer(locale = "pt_BR") %>%
  tab_source_note(
    source_note = "Fonte: Dados da TISS (Amostra do Espírito Santo, 1º Trimestre de 2023)"
  ) %>%
  tab_options(source_notes.font.size = px(10))

# 2. Análise por Porte das Operadoras

Esta seção analisa o número de internações estratificado pelo porte das operadoras de plano de saúde e pela distribuição por sexo.

In [ ]:
######################################
# Análise de Internações por Porte
######################################

rm(agrupador, tabela_final)

agrupador <- dados_1T_2023 %>%
  group_by(ID_EVENTO_ATENCAO_SAUDE) %>%
  summarise(
    ID_PLANO = max(ID_PLANO),
    PORTE = max(PORTE),
    FAIXA_ETARIA = max(FAIXA_ETARIA),
    SEXO = max(SEXO)
  ) %>%
  mutate(
    counter = 1,
    counter_feminino = if_else(SEXO == "Feminino", 1, 0)
  ) %>%
  group_by(ID_PLANO) %>%
  summarise(
    PORTE = max(PORTE),
    internacoes_plano = sum(counter),
    internacoes_femininas = sum(counter_feminino)
  ) %>%
  mutate(internacoes_masculinas = internacoes_plano - internacoes_femininas) %>%
  group_by(PORTE) %>%
  summarise(
    internacoes_plano = sum(internacoes_plano),
    internacoes_masculinas = sum(internacoes_masculinas),
    internacoes_femininas = sum(internacoes_femininas)
  ) %>%
  rename(
    Porte = PORTE,
    `Internações totais` = internacoes_plano,
    Masculinas = internacoes_masculinas,
    Femininas = internacoes_femininas
  ) %>%
  mutate(Porte = str_to_sentence(Porte))

tabela_final <- agrupador

# Exibição
tabela_final %>%
  gt() %>%
  tab_header(
    title = "Matriz de Contagem de Internações",
    subtitle = "Análise cruzada entre Porte da operadora do Plano de Saúde e Internações"
  ) %>%
  tab_style(
    style = cell_text(weight = "bold"),
    locations = cells_title()
  ) %>%
  tab_style(
    style = cell_text(size = px(13.5)),
    locations = cells_body()
  ) %>%
  cols_width(everything() ~ px(132)) %>%
  fmt_integer(locale = "pt_BR") %>%
  tab_source_note(
    source_note = "Fonte: Dados da TISS (Amostra do Espírito Santo, 1º Trimestre de 2023)"
  ) %>%
  tab_options(source_notes.font.size = px(10))

# 3. Análise de Custos Assistenciais

Esta seção explora o Valor Médio do Evento (custo dos procedimentos), variável proxy para o custo assistencial. A análise é estratificada por faixa etária e diagnóstico, permitindo identificar grupos de risco financeiro.

In [ ]:
######################################
# Preparação dos dados de custo
######################################

rm(agrupador, tabela_final)

agrupador <- dados_1T_2023 %>%
  group_by(ID_EVENTO_ATENCAO_SAUDE) %>%
  summarise(
    VL_ITEM_EVENTO_INFORMADO = round(mean(VL_ITEM_EVENTO_INFORMADO), 2),
    FAIXA_ETARIA = max(FAIXA_ETARIA),
    SEXO = max(SEXO),
    ID_CID10 = max(ID_CID10)
  ) %>%
  mutate(
    faixas_idade = case_when(
      FAIXA_ETARIA %in% c("<1", "1 a 4", "5 a 9") ~ "0 a 9",
      FAIXA_ETARIA %in% c("10 a 14", "15 a 19") ~ "10 a 19",
      FAIXA_ETARIA %in% c("20 a 29", "30 a 39") ~ "20 a 39",
      FAIXA_ETARIA %in% c("40 a 49", "50 a 59") ~ "40 a 59",
      FAIXA_ETARIA %in% c("Não identificado") ~ "Não identificado",
      TRUE ~ "60 ou mais"
    )
  ) %>%
  filter(ID_CID10 %in% c("R", "I", "J", "K")) %>%
  mutate(
    Diagnostico = case_when(
      ID_CID10 == 'R' ~ "Achados anormais em exames",
      ID_CID10 == 'I' ~ "Aparelho Circulatório",
      ID_CID10 == 'J' ~ "Aparelho Respiratório",
      TRUE ~ "Aparelho Digestivo"
    )
  ) %>%
  group_by(faixas_idade, Diagnostico) %>%
  summarise(
    VL_ITEM_EVENTO_INFORMADO = round(mean(VL_ITEM_EVENTO_INFORMADO, na.rm = TRUE), 2)
  ) %>%
  pivot_wider(
    names_from = Diagnostico,
    values_from = VL_ITEM_EVENTO_INFORMADO,
    values_fill = 0
  )

tabela_final <- agrupador %>%
  mutate(
    `Achados anormais em exames` = paste0("R$ ", `Achados anormais em exames`),
    `Aparelho Circulatório` = paste0("R$ ", `Aparelho Circulatório`),
    `Aparelho Digestivo` = paste0("R$ ", `Aparelho Digestivo`),
    `Aparelho Respiratório` = paste0("R$ ", `Aparelho Respiratório`)
  ) %>%
  rename(`Faixas de Idade` = faixas_idade)

# Exibição com kable
n_linhas <- nrow(tabela_final)

tabela_final %>%
  mutate(across(everything(), ~ str_replace(., "\\.", ","))) %>%
  kbl(
    caption = "<span style='color: black; font-weight: bold; font-size: 16px'>Custo Médio de Procedimentos por Faixa Etária e Diagnóstico</span>",
    align = c("l", "c", "c", "c", "c")
  ) %>%
  kable_styling(
    bootstrap_options = c("striped", "hover", "condensed", "responsive"),
    full_width = FALSE,
    position = "center",
    font_size = 15
  ) %>%
  row_spec(0, bold = TRUE, color = "white", background = "#023e8a") %>%
  row_spec(n_linhas, extra_css = "border-bottom: 0.5px solid black;") %>%
  column_spec(1, bold = TRUE, border_right = TRUE) %>%
  column_spec(2:5, width = "3.55cm") %>%
  footnote(
    general = "Dados da TISS (Amostra do Espírito Santo, 1º Trimestre de 2023)",
    general_title = "Fonte: ",
    footnote_as_chunk = TRUE,
    title_format = c("italic", "bold"),
    fixed_small_size = TRUE
  )

In [ ]:
######################################
# Gráfico de Barras Empilhadas
######################################

dados_long_tidyr <- agrupador %>%
  pivot_longer(
    cols = c(
      "Achados anormais em exames",
      "Aparelho Circulatório",
      "Aparelho Digestivo",
      "Aparelho Respiratório"
    ),
    names_to = "Diagnostico",
    values_to = "n"
  ) %>%
  group_by(faixas_idade) %>%
  mutate(porcentagem = (n / sum(n))) %>%
  ungroup()

# Gráfico de barras
p1 <- ggplot(dados_long_tidyr, aes(fill = Diagnostico, y = n, x = faixas_idade)) +
  geom_bar(stat = "identity", position = "stack", width = 0.7, color = "white") +
  geom_text(
    aes(label = format(n, decimal.mark = ",", big.mark = ".", nsmall = 2, scientific = FALSE)),
    position = position_stack(vjust = 0.5),
    color = "white",
    fontface = "bold",
    size = 4
  ) +
  scale_fill_viridis_d(option = "D", begin = 0.2, end = 0.8) +
  labs(
    title = "Distribuição de Custo Médio de Procedimentos por Faixa Etária",
    subtitle = "Análise da amostra para o ES, 1º Trimestre de 2023.",
    x = "",
    y = "R$",
    fill = "Diagnóstico",
    caption = "Fonte: Dados da TISS | Elaborado pela equipe do DESID/MS."
  ) +
  theme_minimal(base_size = 14) +
  theme(
    plot.title = element_text(face = "bold", size = 18, color = "#333333"),
    plot.subtitle = element_text(color = "#666666", margin = margin(b = 10)),
    legend.position = "top",
    legend.title = element_text(face = "bold"),
    panel.grid.major.x = element_blank(),
    panel.grid.minor = element_blank(),
    axis.text.x = element_text(face = "bold", color = "black"),
    axis.text.y = element_text(color = "gray60"),
    plot.caption = element_text(hjust = 0, color = "gray50"),
    plot.caption.position = "plot"
  )

print(p1)

In [ ]:
######################################
# Gráfico Percentual
######################################

dados_long_tidyr <- dados_long_tidyr %>%
  filter(faixas_idade != "Não identificado")

# Gráfico de porcentagem
p2 <- ggplot(dados_long_tidyr, aes(x = faixas_idade, y = porcentagem, fill = Diagnostico)) +
  geom_bar(stat = "identity", position = "stack", width = 0.6, color = "white", size = 0.5) +
  geom_text(
    aes(label = scales::percent(porcentagem, accuracy = 0.01, decimal.mark = ",")),
    position = position_stack(vjust = 0.5),
    color = "white",
    fontface = "bold",
    size = 4.5
  ) +
  scale_y_continuous(labels = scales::percent) +
  scale_fill_viridis_d(option = "D", begin = 0.2, end = 0.8) +
  labs(
    title = "Distribuição Percentual das Médias de Custos dos Procedimentos",
    subtitle = "Proporção relativa por faixa etária e diagnóstico",
    x = "",
    y = "%",
    fill = NULL,
    caption = "Fonte: Dados da TISS | Elaborado pela equipe do DESID/MS."
  ) +
  theme_minimal(base_size = 14) +
  theme(
    plot.title = element_text(face = "bold", size = 18, color = "#333333"),
    plot.subtitle = element_text(color = "#666666", margin = margin(b = 10)),
    legend.position = "top",
    legend.title = element_text(face = "bold"),
    panel.grid.major.x = element_blank(),
    panel.grid.minor = element_blank(),
    axis.text.x = element_text(face = "bold", color = "black"),
    axis.text.y = element_text(color = "gray60"),
    plot.caption = element_text(hjust = 0, color = "gray50"),
    plot.caption.position = "plot"
  )

print(p2)

# 4. Tempo de Permanência na Internação

A análise do tempo de permanência é um indicador crucial de eficiência hospitalar e planejamento de recursos.

In [ ]:
######################################
# Tempo Médio de Internação
######################################

rm(agrupador, tabela_final)

agrupador <- dados_1T_2023 %>%
  group_by(ID_EVENTO_ATENCAO_SAUDE) %>%
  summarise(
    FAIXA_ETARIA = max(FAIXA_ETARIA),
    CD_TIPO_INTERNACAO = max(CD_TIPO_INTERNACAO),
    TEMPO_DE_PERMANENCIA = max(TEMPO_DE_PERMANENCIA),
    CD_CARATER_ATENDIMENTO = max(CD_CARATER_ATENDIMENTO)
  ) %>%
  filter(TEMPO_DE_PERMANENCIA > 0) %>%
  mutate(
    faixas_idade = case_when(
      FAIXA_ETARIA %in% c("<1", "1 a 4", "5 a 9") ~ "0 a 9",
      FAIXA_ETARIA %in% c("10 a 14", "15 a 19") ~ "10 a 19",
      FAIXA_ETARIA %in% c("20 a 29", "30 a 39") ~ "20 a 39",
      FAIXA_ETARIA %in% c("40 a 49", "50 a 59") ~ "40 a 59",
      FAIXA_ETARIA %in% c("Não identificado") ~ "Não identificado",
      TRUE ~ "60 ou mais"
    ),
    Carater = case_when(
      CD_CARATER_ATENDIMENTO == 1 ~ "Eletiva",
      TRUE ~ "Urgência/Emergência"
    )
  ) %>%
  group_by(faixas_idade, Carater) %>%
  summarise(
    TEMPO_DE_PERMANENCIA = round(mean(TEMPO_DE_PERMANENCIA, na.rm = TRUE), 2)
  ) %>%
  pivot_wider(
    names_from = Carater,
    values_from = TEMPO_DE_PERMANENCIA,
    values_fill = 0
  )

tabela_final <- agrupador %>%
  rename(`Faixas de Idade` = faixas_idade)

n_linhas <- nrow(tabela_final)

# Exibição
tabela_final %>%
  mutate(across(everything(), ~ str_replace(., "\\.", ","))) %>%
  kbl(
    caption = "<span style='color: black; font-weight: bold; font-size: 16px'>Tempo Médio de Internação por Caráter (em dias)</span>",
    align = c("l", "c", "c")
  ) %>%
  kable_styling(
    bootstrap_options = c("striped", "hover", "condensed", "responsive"),
    full_width = FALSE,
    position = "center",
    font_size = 15
  ) %>%
  row_spec(0, bold = TRUE, color = "white", background = "#023e8a") %>%
  column_spec(1, bold = TRUE, border_right = TRUE) %>%
  column_spec(2:3, width = "3.55cm") %>%
  row_spec(n_linhas, extra_css = "border-bottom: 0.5px solid black;") %>%
  footnote(
    general = "Dados da TISS (Amostra do Espírito Santo, 1º Trimestre de 2023)",
    general_title = "Fonte: ",
    footnote_as_chunk = TRUE,
    title_format = c("italic", "bold"),
    fixed_small_size = TRUE
  )

# Conclusões

Este relatório técnico apresentou uma análise descritiva abrangente dos dados da [translate:Troca de Informação de Saúde Suplementar (TISS)] para o Espírito Santo no 1º trimestre de 2023.

## Principais Achados:

1. **Demanda Hospitalar**: Observa-se maior volume de internações de urgência em tipos cirúrgicos e clínicos.

2. **Fluxo Geográfico**: Significativa proporção de pacientes migrando entre municípios para acesso aos serviços.

3. **Disparidades de Custos**: Procedimentos em pacientes com 60 anos ou mais apresentam custos substancialmente maiores.

4. **Operadoras**: Operadoras de grande porte concentram a maior parte das internações.

5. **Tempo de Permanência**: Idosos com atendimento de urgência apresentam tempos de internação significativamente superiores.

---

**Relatório Técnico Gerado em 24/11/2025**

© 2025 - Uso interno e confidencial.